# Hotspot Prediction with LSTM and Google Earth Engine
This notebook demonstrates collecting weekly hotspot counts, NDVI, and land surface temperature (LST) over a selected region in Northern Thailand and training an LSTM model to predict hotspot activity.


In [ ]:
import ee
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime, timedelta

ee.Authenticate()
ee.Initialize()

# Rectangle representing part of Northern Thailand
roi = ee.Geometry.Rectangle([98.0, 17.0, 100.5, 20.0])

# time range
start = ee.Date('2020-01-01')
end = ee.Date('2021-01-01')

# FIRMS fire hotspots (VIIRS)
fire_fc = (ee.FeatureCollection('FIRMS')
           .filterBounds(roi)
           .filterDate(start, end))

# Generate weekly date list
n_weeks = end.difference(start, 'week').round().getInfo()
weeks = [start.advance(i, 'week') for i in range(n_weeks)]

def get_weekly_stats(week_start_date):
    week_end = ee.Date(week_start_date).advance(1, 'week')
    week_filter = ee.Filter.date(week_start_date, week_end)
    fires_week = fire_fc.filter(week_filter)
    fire_count = fires_week.size()

    # NDVI (MODIS MOD13Q1)
    ndvi_ic = (ee.ImageCollection('MODIS/006/MOD13Q1')
               .select('NDVI')
               .filterBounds(roi)
               .filter(week_filter))
    ndvi = ndvi_ic.mean().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=250,
        maxPixels=1e9
    ).get('NDVI')

    # Land surface temperature (MODIS MOD11A2)
    lst_ic = (ee.ImageCollection('MODIS/061/MOD11A2')
              .select('LST_Day_1km')
              .filterBounds(roi)
              .filter(week_filter))
    lst = lst_ic.mean().multiply(0.02).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000,
        maxPixels=1e9
    ).get('LST_Day_1km')

    return ee.Feature(None, {
        'week_start': ee.Date(week_start_date).format('YYYY-MM-dd'),
        'fire_count': fire_count,
        'ndvi': ndvi,
        'lst': lst
    })

weekly_fc = ee.FeatureCollection([get_weekly_stats(w) for w in weeks])
features = weekly_fc.getInfo()['features']
records = [f['properties'] for f in features]

df = pd.DataFrame(records)
df['week_start'] = pd.to_datetime(df['week_start'])
df = df.sort_values('week_start')
df[['ndvi','lst','fire_count']] = df[['ndvi','lst','fire_count']].astype(float)
df


In [ ]:
# Prepare data for LSTM
features = df[['ndvi','lst','fire_count']].values
look_back = 4  # use previous 4 weeks

X = []
y = []
for i in range(len(features) - look_back):
    X.append(features[i:i+look_back])
    y.append(features[i+look_back, 2])  # predict fire_count

X = np.array(X)
y = np.array(y)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(look_back, 3)),
    tf.keras.layers.LSTM(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, y, epochs=50, batch_size=8, verbose=1)


In [ ]:
# Forecast next week fire hotspot count
last_seq = features[-look_back:]
prediction = model.predict(last_seq.reshape(1, look_back, 3))
print(f'Predicted hotspot count next week: {prediction[0,0]:.2f}')
